# Capstone : Future Housing Prices In Metro Portland, OR Cities

The below is my process of seeing how my data works together, as well as the process for how I learn to model to predict the future prices around Portland Oregon

Import numpy and pandas

In [1]:
import numpy as np
import pandas as pd

create dataframe with tsv document

In [2]:
df = pd.read_csv("city_market_tracker.tsv000", delimiter='\t')

## EDA

we are going to look into my dataset and see how it all the columns relate to eachother

In [3]:
df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2012-08-01,2012-08-31,30,place,6,9148,f,"Kirkland, WA",Kirkland,Washington,...,0.144659,0.193548,-0.003173,-0.153757,0.607143,0.044643,0.496032,"Seattle, WA",42644,2023-02-12 14:58:17
1,2021-11-01,2021-11-30,30,place,6,12172,f,"Marietta, OH",Marietta,Ohio,...,-0.050000,0.152174,0.012174,NaN,0.307692,0.013575,0.307692,"Marietta, OH",31930,2023-02-12 14:58:17
2,2021-12-01,2021-12-31,30,place,6,1378,f,"Belvedere, CA",Belvedere,California,...,0.666667,NaN,NaN,NaN,1.000000,0.000000,0.750000,"San Rafael, CA",42034,2023-02-12 14:58:17
3,2016-10-01,2016-10-31,30,place,6,2341,f,"Butler, NJ",Butler,New Jersey,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,"Newark, NJ",35084,2023-02-12 14:58:17
4,2015-08-01,2015-08-31,30,place,6,11486,f,"Lindon, UT",Lindon,Utah,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Provo, UT",39340,2023-02-12 14:58:17


In [4]:
# look at all columns and column types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4616565 entries, 0 to 4616564
Data columns (total 58 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   period_begin                    object 
 1   period_end                      object 
 2   period_duration                 int64  
 3   region_type                     object 
 4   region_type_id                  int64  
 5   table_id                        int64  
 6   is_seasonally_adjusted          object 
 7   region                          object 
 8   city                            object 
 9   state                           object 
 10  state_code                      object 
 11  property_type                   object 
 12  property_type_id                int64  
 13  median_sale_price               float64
 14  median_sale_price_mom           float64
 15  median_sale_price_yoy           float64
 16  median_list_price               float64
 17  median_list_price_mom      

We want to view only the the cities I am looking into around the Portland area

It looks like the data can provide that, uner `parent_metro_region`

lets filter this and see

In [5]:
df_or = df.loc[df["parent_metro_region"]=="Portland, OR"]

In [6]:
df_or.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
133,2013-10-01,2013-10-31,30,place,6,26540,f,"West Haven-Sylvan, OR",West Haven-Sylvan,Oregon,...,0.000000,0.666667,0.500000,0.166667,NaN,NaN,NaN,"Portland, OR",38900,2023-02-12 14:58:17
598,2013-09-01,2013-09-30,30,place,6,36337,f,"Rhododendron, OR",Rhododendron,Oregon,...,0.500000,0.263158,0.063158,-0.014620,0.0,-1.00,-0.500000,"Portland, OR",38900,2023-02-12 14:58:17
602,2015-05-01,2015-05-31,30,place,6,6130,f,"Estacada, OR",Estacada,Oregon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Portland, OR",38900,2023-02-12 14:58:17
620,2016-05-01,2016-05-31,30,place,6,7385,f,"Gladstone, OR",Gladstone,Oregon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Portland, OR",38900,2023-02-12 14:58:17
685,2012-05-01,2012-05-31,30,place,6,18977,f,"Troutdale, OR",Troutdale,Oregon,...,0.158088,0.250000,0.041667,-0.030000,0.3,0.05,-0.033333,"Portland, OR",38900,2023-02-12 14:58:17


It looks like it worked, but lets check all the cities

In [7]:
# gives all unique cities
df_or["city"].unique()

array(['West Haven-Sylvan', 'Rhododendron', 'Estacada', 'Gladstone',
       'Troutdale', 'Barberton', 'Dundee', 'Sherwood', 'Raleigh Hills',
       'Mulino', 'Cedar Mill', 'Hazel Dell', 'Mount Hood Villages',
       'Ridgefield', 'Vernonia', 'Scappoose', 'West Slope', 'Tualatin',
       'Maywood Park', 'Amboy', 'Happy Valley', 'West Linn',
       'Fern Prairie', 'Oak Grove', 'Washougal', 'Oatfield',
       'Mount Hood Village', 'King City', 'Rockcreek', 'Felida', 'Boring',
       'Clackamas', 'Camas', 'Warren', 'Mill Plain', 'Oregon City',
       'Rainier', 'Damascus', 'Oak Hills', 'Brush Prairie', 'Vancouver',
       'Metzger', 'Carlton', 'Marlene Village', 'Yamhill', 'Beaverton',
       'Willamina', 'Yacolt', 'Cherry Grove', 'Durham', 'Venersborg',
       'Gresham', 'McMinnville', 'Portland', 'Lake Oswego', 'Beavercreek',
       'Rivergrove', 'Tigard', 'Orchards', 'Fairview', 'Amity', 'Bethany',
       'Dollars Corner', 'Aloha', 'Newberg', 'Sunnyside', 'Lafayette',
       'North Plai

Some Cities I don't recognize. Lets see if all are in Oregon. 

We only want to deal with the cities in Oregon.

In [8]:
df_or["state"].unique()

array(['Oregon', 'Washington'], dtype=object)

Looks like it does include cities not in Oregon

In [9]:
df_or = df_or.loc[df_or["state"]=="Oregon"]

In [10]:
df_or["state"].unique()

array(['Oregon'], dtype=object)

In [11]:
df_or.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
133,2013-10-01,2013-10-31,30,place,6,26540,f,"West Haven-Sylvan, OR",West Haven-Sylvan,Oregon,...,0.000000,0.666667,0.500000,0.166667,NaN,NaN,NaN,"Portland, OR",38900,2023-02-12 14:58:17
598,2013-09-01,2013-09-30,30,place,6,36337,f,"Rhododendron, OR",Rhododendron,Oregon,...,0.500000,0.263158,0.063158,-0.014620,0.0,-1.00,-0.500000,"Portland, OR",38900,2023-02-12 14:58:17
602,2015-05-01,2015-05-31,30,place,6,6130,f,"Estacada, OR",Estacada,Oregon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Portland, OR",38900,2023-02-12 14:58:17
620,2016-05-01,2016-05-31,30,place,6,7385,f,"Gladstone, OR",Gladstone,Oregon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Portland, OR",38900,2023-02-12 14:58:17
685,2012-05-01,2012-05-31,30,place,6,18977,f,"Troutdale, OR",Troutdale,Oregon,...,0.158088,0.250000,0.041667,-0.030000,0.3,0.05,-0.033333,"Portland, OR",38900,2023-02-12 14:58:17


In [12]:
df_or.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27753 entries, 133 to 4616503
Data columns (total 58 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   period_begin                    27753 non-null  object 
 1   period_end                      27753 non-null  object 
 2   period_duration                 27753 non-null  int64  
 3   region_type                     27753 non-null  object 
 4   region_type_id                  27753 non-null  int64  
 5   table_id                        27753 non-null  int64  
 6   is_seasonally_adjusted          27753 non-null  object 
 7   region                          27753 non-null  object 
 8   city                            27753 non-null  object 
 9   state                           27753 non-null  object 
 10  state_code                      27753 non-null  object 
 11  property_type                   27753 non-null  object 
 12  property_type_id            

Filtered Dataset shows it has 57 columns and 27753 rows. 

to make things easier with this dataset, we will change the index number to match the leinth of the current dataset

this might change later

In [13]:
# will sort by period_begin
df_or = df_or.sort_values(by="period_begin")
df_or.index = range(len(df_or))

In [14]:
df_or.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2012-01-01,2012-01-31,30,place,6,13439,f,"Newberg, OR",Newberg,Oregon,...,-0.083333,0.329545,0.116062,0.112985,0.200000,-0.466667,-0.244444,"Portland, OR",38900,2023-02-12 14:58:17
1,2012-01-01,2012-01-31,30,place,6,2767,f,"Canby, OR",Canby,Oregon,...,0.200000,0.271186,0.008029,-0.047962,0.285714,-0.047619,-0.047619,"Portland, OR",38900,2023-02-12 14:58:17
2,2012-01-01,2012-01-31,30,place,6,36348,f,"Boring, OR",Boring,Oregon,...,0.000000,0.125000,0.013889,-0.208333,NaN,NaN,NaN,"Portland, OR",38900,2023-02-12 14:58:17
3,2012-01-01,2012-01-31,30,place,6,21950,f,"Clackamas, OR",Clackamas,Oregon,...,-0.083333,0.151515,0.031515,-0.260250,0.500000,0.000000,0.500000,"Portland, OR",38900,2023-02-12 14:58:17
4,2012-01-01,2012-01-31,30,place,6,8192,f,"Happy Valley, OR",Happy Valley,Oregon,...,-0.125000,0.145251,-0.017399,-0.059718,0.250000,-0.035714,-0.083333,"Portland, OR",38900,2023-02-12 14:58:17


We will now look at the null values and see if there are any

In [15]:
df_or.isnull().sum()

period_begin                         0
period_end                           0
period_duration                      0
region_type                          0
region_type_id                       0
table_id                             0
is_seasonally_adjusted               0
region                               0
city                                 0
state                                0
state_code                           0
property_type                        0
property_type_id                     0
median_sale_price                    0
median_sale_price_mom             3220
median_sale_price_yoy             3521
median_list_price                 3192
median_list_price_mom             5530
median_list_price_yoy             5879
median_ppsf                         28
median_ppsf_mom                   3233
median_ppsf_yoy                   3534
median_list_ppsf                  3194
median_list_ppsf_mom              5534
median_list_ppsf_yoy              5879
homes_sold               

There seems to be a fair amount of null values.

Some of these columns though do not seem neccessary. 

I believe we can remove any mom and yoy column as they are interpreted by calculations in the current dataset.

This means there is likely to be colinearity down the line within each. 

In [16]:
# looks for columns with yoy or mom in it and drops them
df_or = df_or.drop(columns=df_or.columns[df_or.columns.str.contains('yoy|mom')])

In [17]:
# Sanity Check
df_or.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753 entries, 0 to 27752
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   period_begin                    27753 non-null  object 
 1   period_end                      27753 non-null  object 
 2   period_duration                 27753 non-null  int64  
 3   region_type                     27753 non-null  object 
 4   region_type_id                  27753 non-null  int64  
 5   table_id                        27753 non-null  int64  
 6   is_seasonally_adjusted          27753 non-null  object 
 7   region                          27753 non-null  object 
 8   city                            27753 non-null  object 
 9   state                           27753 non-null  object 
 10  state_code                      27753 non-null  object 
 11  property_type                   27753 non-null  object 
 12  property_type_id                

Before we go back and try to handle the nulls, are there any columns are not needed?

We will look and see if any columns are just not usefull, or hold irrelevant data.

In [18]:
df_or["region_type"]

0        place
1        place
2        place
3        place
4        place
         ...  
27748    place
27749    place
27750    place
27751    place
27752    place
Name: region_type, Length: 27753, dtype: object

In [19]:
df_or.loc[df_or["region_type"]!="place"]

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated


In [20]:
df_or["region_type_id"]

0        6
1        6
2        6
3        6
4        6
        ..
27748    6
27749    6
27750    6
27751    6
27752    6
Name: region_type_id, Length: 27753, dtype: int64

In [21]:
df_or.loc[df_or["region_type_id"]!=6]

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated


`region_type` through out all the rows only shows "Place"

`region_type_id` through out all the rows only shows 6

We will drop these columns as they don't hold anything that is different between each row

In [22]:
df_or = df_or.drop(columns="region_type")

In [23]:
df_or = df_or.drop(columns="region_type_id")

In [24]:
# sanity
df_or.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753 entries, 0 to 27752
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   period_begin                    27753 non-null  object 
 1   period_end                      27753 non-null  object 
 2   period_duration                 27753 non-null  int64  
 3   table_id                        27753 non-null  int64  
 4   is_seasonally_adjusted          27753 non-null  object 
 5   region                          27753 non-null  object 
 6   city                            27753 non-null  object 
 7   state                           27753 non-null  object 
 8   state_code                      27753 non-null  object 
 9   property_type                   27753 non-null  object 
 10  property_type_id                27753 non-null  int64  
 11  median_sale_price               27753 non-null  float64
 12  median_list_price               

In [25]:
df_or.head()

,period_begin,period_end,period_duration,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2012-01-01,2012-01-31,30,13439,f,"Newberg, OR",Newberg,Oregon,OR,Single Family Residential,...,88.0,6.3,85.0,0.953472,0.000,0.329545,0.200000,"Portland, OR",38900,2023-02-12 14:58:17
1,2012-01-01,2012-01-31,30,2767,f,"Canby, OR",Canby,Oregon,OR,Single Family Residential,...,59.0,5.9,135.0,0.943829,0.200,0.271186,0.285714,"Portland, OR",38900,2023-02-12 14:58:17
2,2012-01-01,2012-01-31,30,36348,f,"Boring, OR",Boring,Oregon,OR,All Residential,...,8.0,2.7,164.0,0.980100,0.000,0.125000,NaN,"Portland, OR",38900,2023-02-12 14:58:17
3,2012-01-01,2012-01-31,30,21950,f,"Clackamas, OR",Clackamas,Oregon,OR,All Residential,...,33.0,8.3,155.0,0.859180,0.250,0.151515,0.500000,"Portland, OR",38900,2023-02-12 14:58:17
4,2012-01-01,2012-01-31,30,8192,f,"Happy Valley, OR",Happy Valley,Oregon,OR,All Residential,...,179.0,7.5,142.0,0.964494,0.125,0.145251,0.250000,"Portland, OR",38900,2023-02-12 14:58:17


I believe the following columns have the same issue. I will look into them and remove if so.
- `period_duration`
- `is_seasonally_adjusted`

The others below should be obvious as well
- `state` 
- `parent_metro_region_metro_code`
- `parent_metro_region`

In [26]:
# period_duration count
df_or["period_duration"].loc[df_or["period_duration"]==30].count()

27753

In [27]:
df_or.loc[df_or["period_duration"]!=30]

,period_begin,period_end,period_duration,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated


In [28]:
# is_seasonally_adjusted count 
df_or["is_seasonally_adjusted"].loc[df_or["is_seasonally_adjusted"]=="f"].count()

27753

In [29]:
df_or.loc[df_or["is_seasonally_adjusted"]!="f"]

,period_begin,period_end,period_duration,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated


In [30]:
# state count 
df_or["state"].loc[df_or["state"]=="Oregon"].count()

27753

In [31]:
df_or.loc[df_or["state"]!="Oregon"]

,period_begin,period_end,period_duration,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated


In [32]:
# parent_metro_region_metro_code count 
df_or["parent_metro_region_metro_code"].loc[df_or["parent_metro_region_metro_code"]==38900].count()

27753

In [33]:
df_or.loc[df_or["parent_metro_region_metro_code"]!=38900]

,period_begin,period_end,period_duration,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated


In [34]:
# parent_metro_region count 
df_or["parent_metro_region"].loc[df_or["parent_metro_region"]=="Portland, OR"].count()

27753

In [35]:
df_or.loc[df_or["parent_metro_region"]!="Portland, OR"]

,period_begin,period_end,period_duration,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,...,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,parent_metro_region,parent_metro_region_metro_code,last_updated


All from before show the exact same for all rows, so we will drop them.

In [36]:
 df_or = df_or.drop(columns=["period_duration","is_seasonally_adjusted","state",\
                    "parent_metro_region_metro_code", "parent_metro_region"])

In [37]:
df_or.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753 entries, 0 to 27752
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   period_begin             27753 non-null  object 
 1   period_end               27753 non-null  object 
 2   table_id                 27753 non-null  int64  
 3   region                   27753 non-null  object 
 4   city                     27753 non-null  object 
 5   state_code               27753 non-null  object 
 6   property_type            27753 non-null  object 
 7   property_type_id         27753 non-null  int64  
 8   median_sale_price        27753 non-null  float64
 9   median_list_price        24561 non-null  float64
 10  median_ppsf              27725 non-null  float64
 11  median_list_ppsf         24559 non-null  float64
 12  homes_sold               27753 non-null  float64
 13  pending_sales            23847 non-null  float64
 14  new_listings          

In [38]:
df_or.head()

,period_begin,period_end,table_id,region,city,state_code,property_type,property_type_id,median_sale_price,median_list_price,...,pending_sales,new_listings,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,last_updated
0,2012-01-01,2012-01-31,13439,"Newberg, OR",Newberg,OR,Single Family Residential,6,203500.0,194000.0,...,10.0,22.0,88.0,6.3,85.0,0.953472,0.000,0.329545,0.200000,2023-02-12 14:58:17
1,2012-01-01,2012-01-31,2767,"Canby, OR",Canby,OR,Single Family Residential,6,207888.5,212000.0,...,7.0,12.0,59.0,5.9,135.0,0.943829,0.200,0.271186,0.285714,2023-02-12 14:58:17
2,2012-01-01,2012-01-31,36348,"Boring, OR",Boring,OR,All Residential,-1,315000.0,NaN,...,NaN,NaN,8.0,2.7,164.0,0.980100,0.000,0.125000,NaN,2023-02-12 14:58:17
3,2012-01-01,2012-01-31,21950,"Clackamas, OR",Clackamas,OR,All Residential,-1,78750.0,196900.0,...,4.0,14.0,33.0,8.3,155.0,0.859180,0.250,0.151515,0.500000,2023-02-12 14:58:17
4,2012-01-01,2012-01-31,8192,"Happy Valley, OR",Happy Valley,OR,All Residential,-1,353750.0,334900.0,...,16.0,47.0,179.0,7.5,142.0,0.964494,0.125,0.145251,0.250000,2023-02-12 14:58:17


There is still more to look into. 

`region` is easy to see gives the same result as `city` and `state`. This will be dropped

`last_updated` does not matter to the rest of the data. It is just when this data was edited. 

We will also see if `property_type` is equal to `property_type_id`. 
   - If it is, we will only keep one of them

In [39]:
# drop region & last_update
df_or = df_or.drop(columns=["region", "last_updated"])

In [40]:
df_or.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753 entries, 0 to 27752
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   period_begin             27753 non-null  object 
 1   period_end               27753 non-null  object 
 2   table_id                 27753 non-null  int64  
 3   city                     27753 non-null  object 
 4   state_code               27753 non-null  object 
 5   property_type            27753 non-null  object 
 6   property_type_id         27753 non-null  int64  
 7   median_sale_price        27753 non-null  float64
 8   median_list_price        24561 non-null  float64
 9   median_ppsf              27725 non-null  float64
 10  median_list_ppsf         24559 non-null  float64
 11  homes_sold               27753 non-null  float64
 12  pending_sales            23847 non-null  float64
 13  new_listings             24561 non-null  float64
 14  inventory             

In [41]:
df_or[['property_type','property_type_id']].iloc[0:15]

,property_type,property_type_id
0,Single Family Residential,6
1,Single Family Residential,6
2,All Residential,-1
3,All Residential,-1
4,All Residential,-1
5,Single Family Residential,6
6,Single Family Residential,6
7,Single Family Residential,6
8,Townhouse,13
9,All Residential,-1


`property_type` is equal to `property_type_id` and we will drop `property_type_id`

In [42]:
df_or = df_or.drop(columns=["property_type_id"])

In [43]:
df_or.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753 entries, 0 to 27752
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   period_begin             27753 non-null  object 
 1   period_end               27753 non-null  object 
 2   table_id                 27753 non-null  int64  
 3   city                     27753 non-null  object 
 4   state_code               27753 non-null  object 
 5   property_type            27753 non-null  object 
 6   median_sale_price        27753 non-null  float64
 7   median_list_price        24561 non-null  float64
 8   median_ppsf              27725 non-null  float64
 9   median_list_ppsf         24559 non-null  float64
 10  homes_sold               27753 non-null  float64
 11  pending_sales            23847 non-null  float64
 12  new_listings             24561 non-null  float64
 13  inventory                25490 non-null  float64
 14  months_of_supply      

In [44]:
df_or

,period_begin,period_end,table_id,city,state_code,property_type,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks
0,2012-01-01,2012-01-31,13439,Newberg,OR,Single Family Residential,203500.0,194000.0,113.102272,129.388295,14.0,10.0,22.0,88.0,6.3,85.0,0.953472,0.000000,0.329545,0.200000
1,2012-01-01,2012-01-31,2767,Canby,OR,Single Family Residential,207888.5,212000.0,118.160670,117.776639,10.0,7.0,12.0,59.0,5.9,135.0,0.943829,0.200000,0.271186,0.285714
2,2012-01-01,2012-01-31,36348,Boring,OR,All Residential,315000.0,NaN,121.621622,NaN,3.0,NaN,NaN,8.0,2.7,164.0,0.980100,0.000000,0.125000,NaN
3,2012-01-01,2012-01-31,21950,Clackamas,OR,All Residential,78750.0,196900.0,56.951548,115.415213,4.0,4.0,14.0,33.0,8.3,155.0,0.859180,0.250000,0.151515,0.500000
4,2012-01-01,2012-01-31,8192,Happy Valley,OR,All Residential,353750.0,334900.0,114.373978,124.968695,24.0,16.0,47.0,179.0,7.5,142.0,0.964494,0.125000,0.145251,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27748,2023-01-01,2023-01-31,10280,Lafayette,OR,Single Family Residential,480477.0,474960.0,275.438249,255.767367,6.0,3.0,3.0,12.0,2.0,21.0,0.991811,0.166667,0.750000,0.000000
27749,2023-01-01,2023-01-31,1036,Banks,OR,All Residential,420000.0,NaN,290.055249,NaN,3.0,2.0,NaN,1.0,0.3,9.0,0.980415,0.000000,1.000000,0.000000
27750,2023-01-01,2023-01-31,30773,Wilsonville,OR,Single Family Residential,632000.0,769000.0,296.333003,308.143921,15.0,11.0,16.0,34.0,2.3,48.0,0.971953,0.000000,0.352941,0.454545
27751,2023-01-01,2023-01-31,31646,Bethany,OR,Single Family Residential,875000.0,830000.0,324.910434,332.672581,17.0,18.0,22.0,49.0,2.9,7.0,0.994274,0.352941,0.224490,0.500000


Looking at what we have left:

`state_code` just lists the state, so we can remove that

`table_id` this gives the original table ID, which is not needed now

we will drop both

In [45]:
df_or = df_or.drop(columns=["state_code", "table_id"])

In [46]:
df_or.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753 entries, 0 to 27752
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   period_begin             27753 non-null  object 
 1   period_end               27753 non-null  object 
 2   city                     27753 non-null  object 
 3   property_type            27753 non-null  object 
 4   median_sale_price        27753 non-null  float64
 5   median_list_price        24561 non-null  float64
 6   median_ppsf              27725 non-null  float64
 7   median_list_ppsf         24559 non-null  float64
 8   homes_sold               27753 non-null  float64
 9   pending_sales            23847 non-null  float64
 10  new_listings             24561 non-null  float64
 11  inventory                25490 non-null  float64
 12  months_of_supply         25490 non-null  float64
 13  median_dom               27587 non-null  float64
 14  avg_sale_to_list      

Now that we trimmed the fat, lets take a look again at the null values

In [47]:
df_or.isnull().sum()

period_begin                  0
period_end                    0
city                          0
property_type                 0
median_sale_price             0
median_list_price          3192
median_ppsf                  28
median_list_ppsf           3194
homes_sold                    0
pending_sales              3906
new_listings               3192
inventory                  2263
months_of_supply           2263
median_dom                  166
avg_sale_to_list              7
sold_above_list               0
price_drops                6388
off_market_in_two_weeks    3906
dtype: int64

My first thought was to see if there is any specific `property_type` or `city` that creates the majority of the nulls

I created for loops to look into both

In [48]:
unique_property_type = df_or['property_type'].unique()

# loop over each property_type and check for null values
for ptype in unique_property_type:
    city_df = df_or[df_or['property_type'] == ptype]
    null_columns = city_df.isnull().sum()
    print(f"{ptype} {null_columns}")
    print("   ")


Single Family Residential period_begin                  0
period_end                    0
city                          0
property_type                 0
median_sale_price             0
median_list_price           455
median_ppsf                   0
median_list_ppsf            456
homes_sold                    0
pending_sales               698
new_listings                455
inventory                   184
months_of_supply            184
median_dom                   21
avg_sale_to_list              0
sold_above_list               0
price_drops                1056
off_market_in_two_weeks     698
dtype: int64
   
All Residential period_begin                  0
period_end                    0
city                          0
property_type                 0
median_sale_price             0
median_list_price           406
median_ppsf                   1
median_list_ppsf            407
homes_sold                    0
pending_sales               657
new_listings                406
inventory    

In [49]:
unique_cities = df_or['city'].unique()

# loop over each city and check for null values in that city
for city in unique_cities:
    city_df = df_or[df_or['city'] == city]
    null_columns = city_df.isnull().sum().max()
    total_rows = len(city_df.T.count())
    thing = null_columns/total_rows
    if null_columns:
        print(f"{thing} {city}")
    else:
        print(f"No null values found in {total_columns} columns for city {city}")

0.3253968253968254 Newberg
0.3033175355450237 Canby
0.33488372093023255 Boring
0.24188790560471976 Clackamas
0.1471861471861472 Happy Valley
0.3333333333333333 McMinnville
0.13064516129032258 Beaverton
0.12267657992565056 Vernonia
0.21855670103092784 West Haven-Sylvan
0.2774869109947644 West Slope
0.1073170731707317 Hillsboro
0.2891832229580574 Sunnyside
0.2608695652173913 King City
0.04 Estacada
0.2684085510688836 Sandy
0.24444444444444444 Cornelius
0.16837782340862423 Bethany
0.298050139275766 Rockcreek
0.2577777777777778 Milwaukie
0.2950191570881226 Banks
0.14556962025316456 Oatfield
0.20249221183800623 Garden Home-Whitford
0.2375886524822695 Aloha
0.31955922865013775 Marlene Village
0.12631578947368421 Rainier
0.2903225806451613 Fairview
0.2606837606837607 Yamhill
0.32083333333333336 Oregon City
0.26649746192893403 St. Helens
0.20833333333333334 Molalla
0.1937984496124031 Columbia City
0.10609037328094302 Wilsonville
0.12313432835820895 Carlton
0.30904522613065327 Troutdale
0.17791

In [50]:
df_or.loc[df_or["city"] == "Barlow"].isnull().sum()

period_begin                0
period_end                  0
city                        0
property_type               0
median_sale_price           0
median_list_price          26
median_ppsf                 0
median_list_ppsf           26
homes_sold                  0
pending_sales              26
new_listings               26
inventory                  26
months_of_supply           26
median_dom                  0
avg_sale_to_list            0
sold_above_list             0
price_drops                32
off_market_in_two_weeks    26
dtype: int64

There is too much to look at so we will just remove columns with too many nulls right now and maybe put them back on later. We will forward fill the columns we can: `median_ppsf`, `median_dom` and `avg_sale_to_list`

In [51]:
df_n = df_or.copy()

In [52]:
df_n = df_n[["period_begin","period_end","city","property_type","median_sale_price",\
             "median_ppsf","median_dom","avg_sale_to_list","sold_above_list" ]]

In [53]:
df_n.head()

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list
0,2012-01-01,2012-01-31,Newberg,Single Family Residential,203500.0,113.102272,85.0,0.953472,0.000
1,2012-01-01,2012-01-31,Canby,Single Family Residential,207888.5,118.160670,135.0,0.943829,0.200
2,2012-01-01,2012-01-31,Boring,All Residential,315000.0,121.621622,164.0,0.980100,0.000
3,2012-01-01,2012-01-31,Clackamas,All Residential,78750.0,56.951548,155.0,0.859180,0.250
4,2012-01-01,2012-01-31,Happy Valley,All Residential,353750.0,114.373978,142.0,0.964494,0.125


In [54]:
df_n.isnull().sum()

period_begin           0
period_end             0
city                   0
property_type          0
median_sale_price      0
median_ppsf           28
median_dom           166
avg_sale_to_list       7
sold_above_list        0
dtype: int64

In [55]:
# loop through all city and property type combinations to fill median_ppsf
for city in df_n['city'].unique():
    for property_type in df_n['property_type'].unique():
        # filter DataFrame by city and property type
        city_df = df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)]

        # fill null values with previous median_ppsf using interpolate option
        city_df['median_ppsf'] = city_df['median_ppsf'].interpolate()

        # update the original DataFrame with the filled values
        df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'median_ppsf'] = city_df['median_ppsf']


/var/folders/ls/lw2cbdkd379b0hl6y5dwmdnr0000gn/T/ipykernel_13170/1358129582.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df['median_ppsf'] = city_df['median_ppsf'].interpolate()


In [56]:
df_n.loc[df_n["median_ppsf"].isnull()]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list
955,2012-05-01,2012-05-31,Estacada,Multi-Family (2-4 Unit),40000.0,NaN,6.0,0.769971,0.0
1106,2012-06-01,2012-06-30,Forest Grove,Multi-Family (2-4 Unit),154300.0,NaN,16.0,1.102931,1.0
1234,2012-07-01,2012-07-31,Canby,Multi-Family (2-4 Unit),132000.0,NaN,NaN,1.000000,0.0
6017,2014-08-01,2014-08-31,King City,Multi-Family (2-4 Unit),1330000.0,NaN,42.0,0.826087,0.0
6331,2014-10-01,2014-10-31,Rhododendron,Multi-Family (2-4 Unit),25000.0,NaN,301.0,0.714286,0.0
7768,2015-05-01,2015-05-31,Bethany,Multi-Family (2-4 Unit),1000000.0,NaN,181.0,0.952381,0.0


**Looking at below, I am finding out when looking at combinations of `city` and `property_types`, they do not go into every single year and month between 2012 and 2023.**

Also it shows that anything that does not fill with the above method only has 1 row, or is the first row, with this `property_type` and `city`

In [77]:
df_n.loc[(df_n["city"]=="Bethany")&(df_n["property_type"]=="Multi-Family (2-4 Unit)")]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list
7768,2015-05-01,2015-05-31,Bethany,Multi-Family (2-4 Unit),1000000.0,NaN,181.0,0.952381,0.0


In [70]:
df_n.loc[(df_n["city"]=="Canby")&(df_n["property_type"]=="Multi-Family (2-4 Unit)")]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list
1234,2012-07-01,2012-07-31,Canby,Multi-Family (2-4 Unit),132000.0,NaN,NaN,1.000000,0.0
1934,2012-10-01,2012-10-31,Canby,Multi-Family (2-4 Unit),250000.0,60.096154,123.0,1.041667,1.0
2107,2012-11-01,2012-11-30,Canby,Multi-Family (2-4 Unit),165000.0,87.394068,62.0,0.733659,0.0
2963,2013-04-01,2013-04-30,Canby,Multi-Family (2-4 Unit),250000.0,97.770825,20.0,0.980392,0.0
3588,2013-07-01,2013-07-31,Canby,Multi-Family (2-4 Unit),190000.0,86.757991,37.0,0.954774,0.0
5185,2014-04-01,2014-04-30,Canby,Multi-Family (2-4 Unit),269550.0,127.292008,386.0,1.024185,1.0
7214,2015-03-01,2015-03-31,Canby,Multi-Family (2-4 Unit),299900.0,103.413793,235.0,1.000000,0.0
7534,2015-04-01,2015-04-30,Canby,Multi-Family (2-4 Unit),281100.0,107.126524,18.0,0.956773,0.0
8313,2015-08-01,2015-08-31,Canby,Multi-Family (2-4 Unit),239900.0,89.648729,5.0,1.000000,0.0
9111,2015-11-01,2015-11-30,Canby,Multi-Family (2-4 Unit),495000.0,149.456522,45.0,1.000000,0.0


As there is not much left with null data, we will backfill the nulls that have more data for their respective `city` and `property_types`.
- There is not many rows so this shouldn't effect the model we make too much, as well it will allow us to keep all the respective rows. 


 
For any nulls that occur where there is only one row of data for their respective `city` and `property_types`, we will give it the median of that column.
- giving the mean of the column will at least not skew the model in any specific direction.

In [83]:
# loop through all city and property type combinations to fill median_ppsf
for city in df_n['city'].unique():
    for property_type in df_n['property_type'].unique():
        
        if df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)].count().max() > 1:
            
            # filter DataFrame by city and property type
            city_df = df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)]

            # fill null values with previous median_dom using the interpolate option
            city_df['median_ppsf'] = city_df['median_ppsf'].fillna(method="bfill")

            # update the original DataFrame with the filled values
            df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'median_ppsf'] = \
            city_df['median_ppsf']
            
        # if city and property_type have only 1 row, it will make median_ppsf equal the mean of itss column
        else: df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'median_ppsf'] = \
            df_n["median_ppsf"].mean()


/var/folders/ls/lw2cbdkd379b0hl6y5dwmdnr0000gn/T/ipykernel_13170/1529675718.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df['median_ppsf'] = city_df['median_ppsf'].fillna(method="bfill")


In [84]:
df_n.loc[df_n["median_ppsf"].isnull()]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list


In [ ]:
# loop through all city and property type combinations to fill median_dom
for city in df_n['city'].unique():
    for property_type in df_n['property_type'].unique():
        # filter DataFrame by city and property type
        city_df = df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)]

        # fill null values with previous median_dom using interpolate option
        city_df['median_dom'] = city_df['median_dom'].interpolate()

        # update the original DataFrame with the filled values
        df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'median_dom'] = city_df['median_dom']


In [59]:
df_n.loc[df_n["median_dom"].isnull()]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list
1234,2012-07-01,2012-07-31,Canby,Multi-Family (2-4 Unit),132000.0,NaN,NaN,1.000000,0.0
1700,2012-09-01,2012-09-30,Milwaukie,Townhouse,252500.0,146.631823,NaN,0.971528,0.0
5858,2014-08-01,2014-08-31,Scappoose,Townhouse,177600.0,108.690330,NaN,1.000000,0.0
6270,2014-10-01,2014-10-31,Jennings Lodge,Townhouse,179900.0,117.275098,NaN,0.975068,0.0
6613,2014-12-01,2014-12-31,Marlene Village,Townhouse,200000.0,135.869565,NaN,1.000000,0.0
6674,2014-12-01,2014-12-31,Happy Valley,Multi-Family (2-4 Unit),595000.0,163.013699,NaN,1.000000,0.0
6719,2014-12-01,2014-12-31,Sunnyside,Multi-Family (2-4 Unit),595000.0,163.013699,NaN,1.000000,0.0
6971,2015-01-01,2015-01-31,Scappoose,Townhouse,179000.0,109.547124,NaN,1.000000,0.0
10687,2016-07-01,2016-07-31,Columbia City,Townhouse,249500.0,147.720545,NaN,0.998000,0.0


In [67]:
df_n.loc[(df_n["city"]=="Columbia City")&(df_n["property_type"]=="Townhouse")]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list
10687,2016-07-01,2016-07-31,Columbia City,Townhouse,249500.0,147.720545,NaN,0.998000,0.0
17533,2019-02-01,2019-02-28,Columbia City,Townhouse,295000.0,175.282234,92.0,0.986622,0.0
22436,2021-01-01,2021-01-31,Columbia City,Townhouse,325000.0,250.965251,3.0,1.001541,1.0


In [86]:
# loop through all city and property type combinations to fill median_dom
for city in df_n['city'].unique():
    for property_type in df_n['property_type'].unique():
        
        if df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)].count().max() > 1:
            
            # filter DataFrame by city and property type
            city_df = df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)]

            # fill null values with previous median_dom using the interpolate option
            city_df['median_dom'] = city_df['median_dom'].fillna(method="bfill")

            # update the original DataFrame with the filled values
            df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'median_dom'] = \
            city_df['median_dom']
            
        # if city and property_type have only 1 row, it will make median_dom equal the mean of itss column
        else: df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'median_dom'] = \
            df_n["median_dom"].mean()

/var/folders/ls/lw2cbdkd379b0hl6y5dwmdnr0000gn/T/ipykernel_13170/1769119186.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df['median_dom'] = city_df['median_dom'].fillna(method="bfill")


In [87]:
df_n.loc[df_n["median_dom"].isnull()]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list


In [61]:
# loop through all city and property type combinations to fill avg_sale_to_list
for city in df_n['city'].unique():
    for property_type in df_n['property_type'].unique():
        # filter DataFrame by city and property type
        city_df = df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)]

        # fill null values with previous avg_sale_to_list using the interpolate option
        city_df['avg_sale_to_list'] = city_df['avg_sale_to_list'].interpolate()

        # update the original DataFrame with the filled values
        df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'avg_sale_to_list'] = city_df['avg_sale_to_list']


/var/folders/ls/lw2cbdkd379b0hl6y5dwmdnr0000gn/T/ipykernel_13170/2246167043.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df['avg_sale_to_list'] = city_df['avg_sale_to_list'].interpolate()


In [62]:
df_n.loc[df_n["avg_sale_to_list"].isnull()]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list
8818,2015-10-01,2015-10-31,Raleigh Hills,Multi-Family (2-4 Unit),32750000.0,7943.24521,48.0,NaN,0.0


In [88]:
# loop through all city and property type combinations to fill avg_sale_to_list
for city in df_n['city'].unique():
    for property_type in df_n['property_type'].unique():
        
        if df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)].count().max() > 1:
            
            # filter DataFrame by city and property type
            city_df = df_n[(df_n['city'] == city) & (df_n['property_type'] == property_type)]

            # fill null values with previous median_dom using the interpolate option
            city_df['avg_sale_to_list'] = city_df['avg_sale_to_list'].fillna(method="bfill")

            # update the original DataFrame with the filled values
            df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'avg_sale_to_list'] = \
            city_df['avg_sale_to_list']
            
        # if city and property_type have only 1 row, it will make median_dom equal the mean of itss column
        else: df_n.loc[(df_n['city'] == city) & (df_n['property_type'] == property_type), 'avg_sale_to_list'] = \
            df_n["avg_sale_to_list"].mean()

/var/folders/ls/lw2cbdkd379b0hl6y5dwmdnr0000gn/T/ipykernel_13170/2528528670.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df['avg_sale_to_list'] = city_df['avg_sale_to_list'].fillna(method="bfill")


In [89]:
df_n.loc[df_n["avg_sale_to_list"].isnull()]

,period_begin,period_end,city,property_type,median_sale_price,median_ppsf,median_dom,avg_sale_to_list,sold_above_list


All the columns have been cleaned, lets take a final check to see if we are missing anything.

In [90]:
df_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753 entries, 0 to 27752
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   period_begin       27753 non-null  object 
 1   period_end         27753 non-null  object 
 2   city               27753 non-null  object 
 3   property_type      27753 non-null  object 
 4   median_sale_price  27753 non-null  float64
 5   median_ppsf        27753 non-null  float64
 6   median_dom         27753 non-null  float64
 7   avg_sale_to_list   27753 non-null  float64
 8   sold_above_list    27753 non-null  float64
dtypes: float64(5), object(4)
memory usage: 1.9+ MB


In [91]:
df_n.isnull().sum()

period_begin         0
period_end           0
city                 0
property_type        0
median_sale_price    0
median_ppsf          0
median_dom           0
avg_sale_to_list     0
sold_above_list      0
dtype: int64